In [32]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
p_ID = df_test['PassengerId']
df_gender = pd.read_csv('gender_submission.csv')
print(df_train['Cabin'].isna().sum())#687 is way too many nulls. 
#Also, the titanic was like a party ship, where people would get blackout drunk and wake up in other people's cabins.
#So, I will drop the Cabin column, since it is irrelevant
df_train = df_train.drop('Cabin',axis=1)
df_test = df_test.drop('Cabin',axis=1)
df_train = df_train.drop('PassengerId',axis=1)#PassengerId is irrelevant
df_test = df_test.drop('PassengerId',axis=1)
df_train = df_train.drop('Embarked',axis = 1)#Where they embarked on the Titanic is irrelevant,
#unless you believe in the butterfly effect
df_test = df_test.drop('Embarked',axis = 1)
print(df_train['Age'].isna().sum())#There are 177 missing age values, which is too many to look up and impute manually.
#However, I believe age is a relevant feature, and I know that missing values is not such a big problem for naive Bayes,
#So, I will do nothing.
df_train = df_train.drop('Name',axis=1)#Name is irrelevant
df_test = df_test.drop('Name',axis=1)
df_train = df_train.drop('Ticket',axis=1)#I think ticket is as irrelevant as Cabin
df_test = df_test.drop('Ticket',axis=1)
#I won't drop 'Fare' because it signifies who is working to pay their way
#I believe people with family aboard may have been more likely to survive, so I won't drop 'SibSp' and 'Parch'
df_train['Sex'].replace(['female','male'],[0,1],inplace=True)#replace sex with binary
df_test['Sex'].replace(['female','male'],[0,1],inplace=True)
df_train

687
177


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000
887,1,1,0,19.0,0,0,30.0000
888,0,3,0,NaN,1,2,23.4500
889,1,1,1,26.0,0,0,30.0000


In [33]:
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [34]:
df_train['Age'].fillna(df_train['Age'].mean(), inplace=True)#replacing null ages with the mean
df_test['Age'].fillna(df_train['Age'].mean(), inplace=True)#replacing null ages in test with the mean from train
df_train=df_train.round(2)

In [35]:
print(df_test['Fare'].isna().sum())#found a null in test 'Fare' column
df_test['Fare'].fillna(df_train['Fare'].mean(), inplace=True)#replacing null fare in test with the mean from train
df_test = df_test.round(2)
print(df_test['Fare'].isna().sum())
df_test

1
0


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,34.5,0,0,7.83
1,3,0,47.0,1,0,7.00
2,2,1,62.0,0,0,9.69
3,3,1,27.0,0,0,8.66
4,3,0,22.0,1,1,12.29
...,...,...,...,...,...,...
413,3,1,29.7,0,0,8.05
414,1,0,39.0,0,0,108.90
415,3,1,38.5,0,0,7.25
416,3,1,29.7,0,0,8.05


In [36]:
y = df_train['Survived']
X = df_train.drop('Survived',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 446 points : 104


In [37]:
def save_preds(_fn, _y_pred, _df):
    import csv
    with open(_fn, 'w') as fout:
        writer = csv.writer(fout, delimiter=',', lineterminator='\n')
        writer.writerow(['PassengerId', 'Survived'])
        for yid, ypred in zip(_df['PassengerId'], _y_pred):
            writer.writerow([yid, ypred])



In [38]:
y_deploy = gnb.fit(X_train, y_train).predict(df_test)
p_ID

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [39]:
save_preds('titanic_predictions.csv', y_deploy, pd.read_csv('test.csv'))